In [8]:
import os

if "notebooks" in os.getcwd():
    os.chdir("..")

In [9]:
import time
from tqdm.notebook import tqdm

In [10]:
from azure_cosmos_db import *

azure_cosmos = AzureCosmos()

azure_cosmos.list_containers()

[INFO] Cosmos client created


['ra_links',
 'eventbrite_events',
 'shotgun_links',
 'ra_events',
 'dice_events',
 'shotgun_events',
 'eventbrite_links']

# Events, Dice & Shotgun Scraper

In [11]:
SCRAPER_NAME = "dice_events"
INPUT_FILE_PATH = "data/inputs/DICE Partner Links - Cleaned - Main.csv"
SHEET_NAME = ""

In [12]:
azure_cosmos.DATABASE_ID = "Scraper"
azure_cosmos.CONTAINER_NAME = SCRAPER_NAME

azure_cosmos.container = azure_cosmos.initialize_cosmosdb()

[INFO] Cosmos client created


In [13]:
SHEET_NAME, extension = os.path.splitext(os.path.basename(INPUT_FILE_PATH))

if extension==".csv":
    df = pd.read_csv(INPUT_FILE_PATH)
elif extension==".xlsx":
    df = pd.read_excel(INPUT_FILE_PATH, sheet_name="Sheet4")

print(f"Total: {df.shape}")
df.drop_duplicates(inplace=True)
print(f"Unique: {df.shape}")

print(SHEET_NAME)
# df = df.sample(5)
df.head()

Total: (59373, 1)
Unique: (59373, 1)
DICE Partner Links - Cleaned - Main


,Source url
0,http://dice.fm/partner/-la-folie/event/229qp-f...
1,http://dice.fm/partner/23-meadow-llc-dba-the-m...
2,http://dice.fm/partner/3615/event/9gww9
3,http://dice.fm/partner/agenda-web/event/3a2ml-...
4,http://dice.fm/partner/agenda-web/event/a39v2-...


In [ ]:
max_retries = 3
success = 0

urls = df["Source url"].to_list()

for url in tqdm(urls, desc=f"Uploading {len(urls)} URLs to {SCRAPER_NAME}"):
    secure_url = url.replace("http://", "https://")
    hash_key = SHEET_NAME + secure_url
    data = {
        "id": hashlib.sha256(hash_key.encode()).hexdigest(),
        "url": secure_url,
        "processed": False,
        "sheet_name": SHEET_NAME
    }

    retries = 0
    while retries < max_retries:
        try:
            azure_cosmos.create_conversation(conversation_data=data)
            success += 1
            break
        except CosmosResourceExistsError as e:
            success += 1
            print(f"[INFO] Record already exists in {azure_cosmos.CONTAINER_NAME}, skipping insertion.")
            break
        except CosmosHttpResponseError as e:
            if e.status_code == 429:
                retry_after = int(e.headers.get("x-ms-retry-after-ms", 1000))/1000
                print(f"[WARNNING] Rate limit exeeded. Retrying in {retry_after}")
                time.sleep(retry_after)
                retries += 1
            else:
                print(e)
                break
                     
print(f"Out of {df.shape[0]} URLs, {success} were uploaded and {df.shape[0]-success} failed to upload")

Uploading 59373 URLs to dice_events:   0%|          | 0/59373 [00:00<?, ?it/s]

[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversat